<a href="https://colab.research.google.com/github/FGalvao77/Performance-entre-as-bibliotecas-Pandas-DuckDB-e-SQLite/blob/main/Performance_entre_as_bibliotecas_Pandas%2C_DuckDB_e_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<br>

## **Performance entre as bibliotecas: `Pandas`, `DuckDB` e `SQLite`**
---
---


- Comparação de **tempo de execução** entre as _libraries_.

In [ ]:
# importando as bibliotecas 
import pandas as pd
import numpy as np

Vamos instanciar um _dataframe_ com dados de duração de corridas de táxi, onde esse conjunto de dados possui 10 milhões de linhas e duas colunas, são elas:
- `pickup_longitude`: longitude de coleta;
- `trip_duration`: duração da viagem.

E depois da criação do conjunto de dados iremos calcular a duração média das corridas que começaram a oeste da longitude -73,95.

In [ ]:
# números de linhas do conjunto de dados
num_rows = 10_000_000

# gerando uma longitude aleatória para cada linha 
pickup_longitude = np.random.uniform(
    low=-38.0, high=-94.0, size=num_rows
)

# gerando uma duração de viagem aleatória
trip_duration = np.random.normal(
    loc=10, scale=5, size=num_rows
)

In [ ]:
# instanciando o dataframe com os dados acima
df = pd.DataFrame(
    {'pickup_longitude': pickup_longitude,
     'trip_duration': trip_duration}
)

In [ ]:
# visualizando as 5 primeiras observações do conjunto de dados 
df.head()

,pickup_longitude,trip_duration
0,-74.039969,13.844311
1,-74.170674,7.925259
2,-90.639128,6.890475
3,-83.348512,7.462734
4,-41.752227,6.159140


In [ ]:
# dimensão do conjunto de dados - linhas e colunas
df.shape

(10000000, 2)

In [ ]:
# importando biblioteca para contablizar tempo
import time

In [ ]:
# criando uma função para realizar o cálculo de execução da consulta do conjunto de dados
def timing_decorator(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        result = func(*args, **kwargs)
        end_time = time.time()

        print(f'Function {func.__name__} took {end_time - start_time} seconds to run.')
        
        return result
    return wrapper

In [ ]:
# utilizando a função "timing_decorator"
@timing_decorator 

# instanciando uma função para exibir o resultado da consulta do tempo médio das corridas
def find_avg_trip_duration_in_the_west():
    return df[df['pickup_longitude'] < -73.95]['trip_duration'].mean()

In [ ]:
# utilizando a função "find_avg_trip_duration_in_the_west()" para visualizar o resultado
find_avg_trip_duration_in_the_west()

Function find_avg_trip_duration_in_the_west took 0.24489307403564453 seconds to run.


9.999314866483957

Vamos instalar a biblioetca [DuckDB](https://duckdb.org/docs/archive/0.2.8/api/python) e utilizar o conjunto de dados acima para realizar a mesma consulta do tempo médio de duração das corridas de táxi com longitude a oeste de 73,95.

In [ ]:
# instalando a biblioteca "duckdb"
%%capture
!pip install duckdb

In [ ]:
# importando a biblioteca
import duckdb

In [ ]:
# utilizando a função "timing_decorator"
@timing_decorator

# instanciando uma função com a biblioetca "DuckDB" para exibir o resultado da consulta do tempo médio das corridas
def find_avg_trip_duration_in_the_west():
    return duckdb.execute(
        '''SELECT AVG(trip_duration)
         FROM df
         WHERE pickup_longitude < -73.95'''
    ).df()

In [ ]:
# utilizando a função "find_avg_trip_duration_in_the_west()" para visualizar o resultado
find_avg_trip_duration_in_the_west()

Function find_avg_trip_duration_in_the_west took 0.13002538681030273 seconds to run.


,avg(trip_duration)
0,9.999315


> Note que, com a utilização da biblioteca `DuckDB` o tempo de execução da consulta foi aproximadamente metade do tempo da biblioteca `Pandas`.

Agora vamos utilizar o `SQLite` e comparar o seu tempo de execução em relação as bibliotecas _Pandas_ e _DuckDB_.

In [ ]:
# importando o "SQLite3"
import sqlite3

In [ ]:
# criando um database e instanciando sua conexão
conn = sqlite3.connect(database='taxi.db')

In [ ]:
# visualizando o database criado
df.to_sql(name='trips', con=conn)

10000000

In [ ]:
# novamente utilizando a função "timing_decorator"
@timing_decorator

# função para exibir o resultado da consulta do tempo médio das corridas 
def find_avg_trip_duration_in_the_west():
    cursor = conn.cursor()
    cursor.execute(
        '''
        SELECT AVG(trip_duration)
        FROM trips 
        WHERE pickup_longitude < -73.95
        '''
    )

    result = cursor.fetchone()[0]
    cursor.close()

    return result

> Ao executar o `SQLite` precisamos criar uma instância separada do banco de dados no disco e insir os dados nela. E todas essas estapas já não é necessária na `DuckDB`.

In [ ]:
# visualizando o resultado de execução da consulta e tempo médio das corridas
find_avg_trip_duration_in_the_west()

Function find_avg_trip_duration_in_the_west took 0.920447587966919 seconds to run.


9.999314866483642

> O tempo de execução foi ainda maior que da biblioteca _Pandas_.

Portanto, através desse experimento prático chegamos à conclusão de que `DuckDB` apresenta o menor tempo de execução que as demais bibliotecas.

Embora a biblioteca `DuckDB` possua vantagens em tempo de execução, isso não é tudo! Você ainda irá precisar de banco de dados escaláveis, talvez como o **Postgres** para projetos de grande escala.